In [11]:
from PIL import Image
import numpy as np

def pack_bw_1bit(image_array):
    flat = image_array.flatten()
    packed = []

    for i in range(0, len(flat), 8):
        byte = 0
        for j in range(8):
            if i + j < len(flat):
                byte |= (flat[i + j] & 1) << (7 - j)
        packed.append(byte)

    return bytes(packed)

def unpack_bw_1bit(packed_data, shape):
    total_pixels = shape[0] * shape[1]
    flat = []

    for byte in packed_data:
        for i in range(7, -1, -1):
            flat.append((byte >> i) & 1)
            if len(flat) == total_pixels:
                break

    return np.array(flat, dtype=np.uint8).reshape(shape)

def rle_encode(data):
    encoded = []
    prev = data[0]
    count = 1

    for byte in data[1:]:
        if byte == prev and count < 255:
            count += 1
        else:
            encoded.extend([count, prev])
            prev = byte
            count = 1

    encoded.extend([count, prev])
    return bytes(encoded)

def rle_decode(encoded):
    decoded = []
    for i in range(0, len(encoded), 2):
        count = encoded[i]
        value = encoded[i + 1]
        decoded.extend([value] * count)
    return bytes(decoded)


# Load strict BW image
bw_image = Image.open("sbw2.png").resize((128, 128)).convert("1")  # 1-bit mode
bw_array = (np.array(bw_image) > 0).astype(np.uint8)  # Ensure 0 and 1 explicitly


# Bit-pack only
packed = pack_bw_1bit(bw_array)
print("📦 Packed Only Size (Bytes):", len(packed))

# # Bit-pack + RLE
# rle_packed = rle_encode(packed)
# print("📦 Packed + RLE Size (Bytes):", len(rle_packed))

# Save .lix file with better strategy
best_data = packed
with open("image_bw.lix", "wb") as f:
    f.write(b'BW1' + bytes([1 if best_data == packed else 0]))  # Simple header
    f.write(best_data)

print(f"✅ Saved .lix file as image_bw.lix, size: {len(best_data)} bytes")


📦 Packed Only Size (Bytes): 2048
✅ Saved .lix file as image_bw.lix, size: 2048 bytes


In [12]:
import numpy as np
from PIL import Image

def unpack_bw_1bit(packed_data, shape):
    total_pixels = shape[0] * shape[1]
    flat = []

    for byte in packed_data:
        for i in range(7, -1, -1):
            flat.append((byte >> i) & 1)
            if len(flat) == total_pixels:
                break

    return np.array(flat, dtype=np.uint8).reshape(shape)

def rle_decode(encoded):
    decoded = []
    for i in range(0, len(encoded), 2):
        count = encoded[i]
        value = encoded[i + 1]
        decoded.extend([value] * count)
    return bytes(decoded)

def decode_bw_lix(filepath, output_png="decoded_bw.png", size=(128, 128)):
    with open(filepath, "rb") as f:
        header = f.read(4)
        if header[:3] != b'BW1':
            raise ValueError("Not a valid BW1 LIX file.")
        
        mode = header[3]  # 1 = bit-packed only, 0 = RLE + packed
        data = f.read()

        if mode == 1:
            packed = data
        elif mode == 0:
            packed = rle_decode(data)
        else:
            raise ValueError("Unknown compression mode.")

    img_array = unpack_bw_1bit(packed, size) * 255
    img = Image.fromarray(img_array.astype(np.uint8), mode="L")
    img.save(output_png)
    print(f"✅ Decoded and saved image as {output_png}")

# Example usage:
decode_bw_lix("image_bw.lix", "decoded_output.png", size=(128, 128))


✅ Decoded and saved image as decoded_output.png


In [16]:
import os

def compare_file_sizes(original_image_path, lix_file_path, decoded_image_path):
    # Get file sizes
    original_size = os.path.getsize(original_image_path)
    lix_size = os.path.getsize(lix_file_path)
    decoded_size = os.path.getsize(decoded_image_path)

    # Print the sizes
    print(f"Original Image Size: {original_size} bytes")
    print(f".lix File Size: {lix_size} bytes")
    print(f"Decoded Image Size: {decoded_size} bytes")

    # Compare sizes
    compression_ratio = original_size / lix_size if lix_size > 0 else float('inf')
    print(f"Compression Ratio (Original / .lix): {compression_ratio:.2f}")

# Example usage:
compare_file_sizes("sbw2.png", "image_bw.lix", "decoded_output.png")

Original Image Size: 1296 bytes
.lix File Size: 2052 bytes
Decoded Image Size: 1296 bytes
Compression Ratio (Original / .lix): 0.63
